In [ ]:
# Reference: https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 11.0MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
     |████████████████████████████████| 3.3MB 40.6MB/s 


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import tensorflow as tf
import pandas as pd

# Task 3: Stack Overflow Question Quality

In [ ]:
# load the first 5000 data from stack overflow question dataset
df2 = pd.read_csv('train.csv')
df2 = df2.drop(columns=['Title', 'Id', 'Tags', 'CreationDate'])
df2 = df2.rename(columns={"Y": "LABEL_COLUMN", "Body": "DATA_COLUMN"})
df2['LABEL_COLUMN'] = df2['LABEL_COLUMN'].replace({'LQ_CLOSE': 0, 'LQ_EDIT': 1, 'HQ': 2})
df2 = df2.head(5000)
df2

,DATA_COLUMN,LABEL_COLUMN
0,<p>I'm already familiar with repeating tasks e...,0
1,<p>I'd like to understand why Java 8 Optionals...,2
2,<p>I am attempting to overlay a title over an ...,2
3,"<p>The question is very simple, but I just cou...",2
4,<p>I'm using custom floatingactionmenu. I need...,2
...,...,...
4995,<p>I would like to perform Spell Checking on t...,0
4996,<p>I have a method:</p>\n\n<pre><code> public...,0
4997,<p>I am facing a weird bug with Facebook SDK. ...,0
4998,<p>How do I get this to not return 'brea outsi...,0


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train2, X_val2 ,y_train2, y_val2 = train_test_split(df2.index.values, df2.LABEL_COLUMN.values, random_state = 2020, test_size = 0.2, stratify=df2.LABEL_COLUMN.values)

In [ ]:
df2['data_type'] = ['not_set']*df2.shape[0]

df2.loc[X_train2, 'data_type'] = 'train'
df2.loc[X_val2, 'data_type'] = 'val'

df2.groupby(['LABEL_COLUMN', 'data_type']).count()

DATA_COLUMN
LABEL_COLUMN data_type             
0            train             1351
             val                338
1            train             1070
             val                268
2            train             1579
             val                394

In [ ]:
# convert text into the encoded form
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train2 = tokenizer.batch_encode_plus(
    df2[df2.data_type=='train'].DATA_COLUMN.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # limit each text input to the first 256 tokens
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val2 = tokenizer.batch_encode_plus(
    df2[df2.data_type=='val'].DATA_COLUMN.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # limit each text input to the first 256 tokens
    max_length=256, 
    return_tensors='pt'
)


input_ids_train2 = encoded_data_train2['input_ids']
attention_masks_train2 = encoded_data_train2['attention_mask']
labels_train2 = torch.tensor(df2[df2.data_type=='train'].LABEL_COLUMN.values)

input_ids_val2 = encoded_data_val2['input_ids']
attention_masks_val2 = encoded_data_val2['attention_mask']
labels_val2 = torch.tensor(df2[df2.data_type=='val'].LABEL_COLUMN.values)

dataset_train2 = TensorDataset(input_ids_train2, attention_masks_train2, labels_train2)
dataset_val2 = TensorDataset(input_ids_val2, attention_masks_val2, labels_val2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# number of labels = 3
# HQ, LQ_EDIT, LQ_CLOSE
model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train2 = DataLoader(dataset_train2, 
                              sampler=RandomSampler(dataset_train2), 
                              batch_size=batch_size)

dataloader_validation2 = DataLoader(dataset_val2, 
                                   sampler=SequentialSampler(dataset_val2), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model2.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train2)*epochs)

In [ ]:
from sklearn.metrics import f1_score

# get weighted f1 score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

# get accuracy
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2.to(device)

print(device)

cuda


In [ ]:
import random
import numpy as np
from sklearn.metrics import accuracy_score

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model2.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model2(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, 3)):
    
    model2.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train2, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model2.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model2(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    print('Epoch ' + str(epoch))
    
    loss_train_avg = loss_train_total/len(dataloader_train2)            
    print('Training loss: ' + str(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation2)
    val_f1 = f1_score_func(predictions, true_vals)
    print('Validation loss: ' + str(val_loss))
    print('F1 Score (Weighted): ' + str(val_f1))
    prediction = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(true_vals, prediction)
    print('Test Accuracy: ' + str(accuracy))


Epoch 1
Training loss: 0.48877048903061815
Validation loss: 0.46984475969635286
F1 Score (Weighted): 0.8646225123156758
Test Accuracy: 0.864


Epoch 2
Training loss: 0.37973670589837427
Validation loss: 0.5055730888392765
F1 Score (Weighted): 0.8835338962530253
Test Accuracy: 0.884



In [ ]:
# precision, recall, f1-score, etc for each class
from sklearn import metrics
print(metrics.classification_report(true_vals, prediction, digits=3))

              precision    recall  f1-score   support

           0      0.846     0.846     0.846       338
           1      1.000     0.996     0.998       268
           2      0.868     0.871     0.869       394

    accuracy                          0.896      1000
   macro avg      0.905     0.904     0.905      1000
weighted avg      0.896     0.896     0.896      1000



# Task 2: AG News Topic Labeling

In [ ]:
# load the first 5000 data from stack overflow question dataset
df = pd.read_csv('news.csv')
df = df.drop(columns=['Title'])
df = df.rename(columns={"Class Index": "LABEL_COLUMN", "Description": "DATA_COLUMN"})
df['LABEL_COLUMN'] = df['LABEL_COLUMN'] - 1
df = df.head(5000)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_val ,y_train, y_val = train_test_split(df.index.values, df.LABEL_COLUMN.values, random_state = 2020, test_size = 0.2, stratify=df.LABEL_COLUMN.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['LABEL_COLUMN', 'data_type']).count()

DATA_COLUMN
LABEL_COLUMN data_type             
0            train              988
             val                247
1            train              826
             val                206
2            train              989
             val                247
3            train             1197
             val                300

In [ ]:
# convert text into the encoded form
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].DATA_COLUMN.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # limit each text input to the first 256 tokens
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].DATA_COLUMN.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    # limit each text input to the first 256 tokens
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].LABEL_COLUMN.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].LABEL_COLUMN.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# number of labels = 4
# World, Sports, Business, Science/Technology
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

# get weighted f1 score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat)

# get accuracy
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
import random
import numpy as np
from sklearn.metrics import accuracy_score

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, 3)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    print('Epoch ' + str(epoch))
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    print('Training loss: ' + str(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    print('Validation loss: ' + str(val_loss))
    print('F1 Score (Weighted): ' + str(val_f1))
    prediction = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(true_vals, prediction)
    print('Test Accuracy: ' + str(accuracy))


Epoch 1
Training loss: 0.5703502880305337
Validation loss: 0.40454724109815265
F1 Score (Weighted): 0.9072529322661016
Test Accuracy: 0.908


Epoch 2
Training loss: 0.30217644879632843
Validation loss: 0.38711408633842626
F1 Score (Weighted): 0.922765235471626
Test Accuracy: 0.923



In [ ]:
# precision, recall, f1-score, etc for each class
from sklearn import metrics
print(metrics.classification_report(true_vals, prediction, digits=3))

              precision    recall  f1-score   support

           0      0.923     0.927     0.925       247
           1      0.961     0.961     0.961       206
           2      0.934     0.858     0.895       247
           3      0.890     0.947     0.918       300

    accuracy                          0.923      1000
   macro avg      0.927     0.923     0.925      1000
weighted avg      0.924     0.923     0.923      1000

